<a href="https://colab.research.google.com/github/Abishekdevs/trinity/blob/main/work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q google-generativeai algoliasearch gspread google-auth

In [3]:
#@title 📍 AI Restaurant Explorer (V3 - Stable) { run: "auto" }
LOCATION_INPUT = "Mahabalipuram, TN, India" #@param {type:"string"}

import requests, json
from google.colab import userdata

def debug_geocode():
    key = userdata.get('MAPS_API_KEY')
    # Using a slightly different URL format for better compatibility
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": LOCATION_INPUT,
        "key": key
    }

    print(f"--- Debugging Connection ---")
    try:
        response = requests.get(url, params=params)
        data = response.json()

        status = data.get("status")
        error_msg = data.get("error_message", "No error message provided.")

        if status == "OK":
            results = data["results"][0]["geometry"]["location"]
            print(f"✅ Success! Found: {results}")
            return results
        else:
            print(f"❌ Status: {status}")
            print(f"❌ Details: {error_msg}")

            if status == "REQUEST_DENIED":
                print("\n💡 ACTION REQUIRED:")
                print("1. Ensure 'Geocoding API' is ENABLED in Cloud Console.")
                print("2. Ensure 'Billing' is linked to this specific project.")
                print("3. Ensure 'Application Restrictions' is set to 'None' in API Key settings.")
    except Exception as e:
        print(f"❌ Critical Error: {e}")

debug_geocode()

--- Debugging Connection ---
✅ Success! Found: {'lat': 12.6207821, 'lng': 80.1944915}


{'lat': 12.6207821, 'lng': 80.1944915}

Sync up, reset _ erase algolia and sheet - updated with recent headers

In [ ]:
#@title 📍Logic to actually wipe the data after confirmation)
Execute_cell_to_get_options = "RUN" #@param {type:"string"}

# 1. ADDITIONAL IMPORTS
import ipywidgets as widgets
from IPython.display import display, clear_output

# 2. UPDATED RESET LOGIC (Confirmation Button Pattern)
def perform_actual_reset():
    """Logic to actually wipe the data after confirmation."""
    with output_log:
        print("🗑️ Resetting GSheets and Algolia...")
        try:
            # 1. Reset GSheets
            sh = gc.open(WORKBOOK_NAME)
            for sheet_name in ["Restaurants", "Menus"]:
                ws = sh.worksheet(sheet_name)
                ws.clear()
                # Restore headers
                if sheet_name == "Restaurants":
                    ws.append_row(['timestamp', 'place_id', 'name', 'phone', 'location', 'city', 'state', 'country', 'cuisine', 'website', 'usp', 'market_gap', 'social_links'])
                else:
                    ws.append_row(['id', 'place_id', 'rest_name', 'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'])

            # 2. Clear Algolia Index
            algolia_client.clear_objects(index_name=ALG_INDEX_NAME)
            print("✨ Everything Reset! Sheets cleared and Algolia index emptied.")
        except Exception as e:
            print(f"❌ Reset Error: {e}")

def on_reset_clicked(b):
    """Triggers the confirmation UI."""
    with output_log:
        clear_output()
        print("⚠️ ARE YOU SURE? This will wipe all data in GSheets and Algolia.")

        # Create confirmation buttons
        confirm_btn = widgets.Button(description="YES, DELETE ALL", button_style='danger')
        cancel_btn = widgets.Button(description="NO, CANCEL", button_style='warning')

        def handle_confirm(b):
            clear_output()
            perform_actual_reset()

        def handle_cancel(b):
            clear_output()
            print("❌ Reset cancelled.")

        confirm_btn.on_click(handle_confirm)
        cancel_btn.on_click(handle_cancel)
        display(widgets.HBox([confirm_btn, cancel_btn]))

# 3. EXISTING SYNC LOGIC
def on_sync_clicked(b):
    with output_log:
        clear_output()
        print("🔄 Starting Full Sync: GSheets -> Algolia...")
        try:
            sh = gc.open(WORKBOOK_NAME)
            records = sh.worksheet("Restaurants").get_all_records()
            if not records:
                print("⚠️ No data in 'Restaurants' sheet to sync.")
                return
            algolia_client.replace_all_objects(index_name=ALG_INDEX_NAME, objects=records)
            res = algolia_client.list_indices()
            count = next((idx.entries for idx in res.items if idx.name == ALG_INDEX_NAME), 0)
            print(f"✅ Sync Complete! Total records in Algolia: {count}")
        except Exception as e:
            print(f"❌ Sync Error: {e}")

# 4. INITIALIZE UI
btn_sync = widgets.Button(description="🔄 Sync Now", button_style='info')
btn_reset = widgets.Button(description="🗑️ Full Reset", button_style='danger')
output_log = widgets.Output()

btn_sync.on_click(on_sync_clicked)
btn_reset.on_click(on_reset_clicked)

print("\n--- 🛠️ CONTROL PANEL ---")
display(widgets.HBox([btn_sync, btn_reset]), output_log)

In [ ]:
#@title 📍 AI Market Intelligence Explorer (v22 - Full Data + Is Unique Locally)
LOCATION_INPUT = "Tambaram, Chennai TN, India" #@param {type:"string"}
RADIUS_METERS = 7000 #@param {type:"slider", min:500, max:50000, step:500}
WORKBOOK_NAME = "V1 Advanced Market Intelligence 2025" #@param {type:"string"}

import requests, json, time, uuid, re
from google import genai
from google.colab import userdata, auth
from google.auth import default
import gspread
from algoliasearch.search.client import SearchClientSync

# --- INITIALIZATION ---
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))
    MAPS_API_KEY = userdata.get('MAPS_API_KEY')
    algolia_client = SearchClientSync(userdata.get('ALGOLIA_APP_ID'), userdata.get('ALGOLIA_API_KEY'))
    ALG_INDEX_NAME = userdata.get('ALGOLIA_INDEX_NAME')

    print("✅ All services Initialized.")
except Exception as e:
    print(f"❌ Setup Error: {e}")

# --- HELPER FUNCTIONS ---

def get_sheets(workbook_name):
    try:
        sh = gc.open(workbook_name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(workbook_name)

    def get_or_create(name, header):
        try:
            ws = sh.worksheet(name)
        except gspread.WorksheetNotFound:
            ws = sh.add_worksheet(name, 1000, len(header))
        if not ws.get_all_values():
            ws.append_row(header)
        return ws

    # Headers for Restaurants
    ws_rest = get_or_create("Restaurants", [
        'timestamp', 'place_id', 'name', 'phone', 'location',
        'city', 'state', 'country', 'cuisine', 'website',
        'usp', 'market_gap', 'social_links'
    ])
    # Headers for Menus (Including is_unique_locally)
    ws_menu = get_or_create("Menus", [
        'id', 'place_id', 'rest_name', 'item_name', 'price',
        'category', 'is_unique_locally', 'competitive_reasoning'
    ])
    return ws_rest, ws_menu

def parse_address_components(components):
    city = state = country = "N/A"
    for c in components:
        types = c.get('types', [])
        if 'locality' in types: city = c.get('longText', 'N/A')
        elif 'administrative_area_level_1' in types: state = c.get('shortText', 'N/A')
        elif 'country' in types: country = c.get('longText', 'N/A')
    return city, state, country

def enrich_market_intelligence(restaurant_name, location):
    prompt = (
        f"Research the restaurant: '{restaurant_name}' at '{location}'.\n"
        "Return ONLY JSON. Rules:\n"
        "- cuisine: Max 3 words.\n"
        "- usp: Max 3 words, catchy caption.\n"
        "- market_gap: Exactly one simple sentence.\n"
        "- menu: get all menu items from the restaurant, out of that, mark 3 unique items with 'competitive_reasoning' relative to local rivals.\n\n"
        "JSON: {'details': {'cuisine', 'website', 'usp', 'market_gap', 'social_links': []}, "
        "'menu': [{'item_name', 'price', 'category', 'is_unique_locally', 'competitive_reasoning'}]}"
    )
    try:
        response = client.models.generate_content(model='gemini-2.0-flash', contents=prompt)
        match = re.search(r'\{.*\}', response.text, re.DOTALL)
        if match:
            data = json.loads(match.group())
            d = data.get('details', {})
            # Text clipping safety
            if d.get('usp'): d['usp'] = " ".join(d['usp'].split()[:3])
            if d.get('cuisine'): d['cuisine'] = " ".join(d['cuisine'].split()[:3])
            return data
    except: return None

# --- MAIN APP ---

def run_app():
    print(f"🚀 Scouting {LOCATION_INPUT}...")
    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)

    # DEDUP: Pull existing Place IDs to avoid redundant API calls
    existing_data = ws_rest.get_all_values()
    existing_ids = {row[1] for row in existing_data[1:]}

    # 1. Geocoding
    geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                       params={"address": LOCATION_INPUT, "key": MAPS_API_KEY}).json()
    if geo['status'] != 'OK': return print("❌ Geocoding Failed.")
    lat, lng = geo['results'][0]['geometry']['location'].values()

    # 2. Nearby Search
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": MAPS_API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
    }
    search_data = {
        "includedTypes": ["restaurant"],
        "maxResultCount": 15,
        "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
    }
    places_resp = requests.post("https://places.googleapis.com/v1/places:searchNearby",
                                headers=headers, json=search_data).json()
    places = places_resp.get('places', [])

    if not places: return print("📍 No restaurants found.")

    all_menu_rows = []
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    for p in places:
        pid = p.get('id')
        name = p['displayName']['text']

        if pid in existing_ids:
            print(f"⏭️ Skipping {name} (Already Audited)")
            continue

        loc = p['formattedAddress']
        phone = p.get('internationalPhoneNumber', 'N/A')
        city, state, country = parse_address_components(p.get('addressComponents', []))

        print(f"🔎 Analyzing: {name}...")
        ai = enrich_market_intelligence(name, loc)
        if not ai: continue

        d, menu_items = ai.get('details', {}), ai.get('menu', [])

        # Save Restaurant Row
        ws_rest.append_row([
            curr_time, pid, name, phone, loc, city, state, country,
            d.get('cuisine'), d.get('website'), d.get('usp'),
            d.get('market_gap'), ", ".join(d.get('social_links', []))
        ])

        # Batch Menu Items (Including is_unique_locally)
        for m in menu_items:
            all_menu_rows.append([
                str(uuid.uuid4()),
                pid,
                name,
                m.get('item_name'),
                m.get('price'),
                m.get('category'),
                m.get('is_unique_locally'), # ADDED BACK
                m.get('competitive_reasoning')
            ])

        # Sync to Algolia
        try:
            algolia_client.save_object(index_name=ALG_INDEX_NAME, body={
                "objectID": pid,
                "name": name,
                "phone": phone,
                "location": loc,
                "city": city,
                "country": country,
                "cuisine": d.get('cuisine'),
                "usp": d.get('usp'),
                "market_gap": d.get('market_gap'),
                "menu": menu_items
            })
        except: pass

        time.sleep(1)

    # Final Batch Upload for Menus
    if all_menu_rows:
        ws_menu.append_rows(all_menu_rows)
        print(f"📊 Added {len(all_menu_rows)} menu items to Sheet.")

    print(f"\n✨ Market Intelligence Audit Complete!")

if __name__ == "__main__":
    run_app()

In [ ]:
# --- ADVANCED BATCH PROCESSOR ---
#@title 📥 Secure Rest Batch Processor (Input from Sheet)
WORKBOOK_NAME = "V1 Advanced Market Intelligence 2025" #@param {type:"string"}
INPUT_TAB_NAME = "Rest_Queue" #@param {type:"string"}

def run_secure_batch_processor():
    print(f"📂 Accessing {WORKBOOK_NAME}...")
    sh = gc.open(WORKBOOK_NAME)

    try:
        ws_input = sh.worksheet(INPUT_TAB_NAME)
    except gspread.WorksheetNotFound:
        return print(f"❌ Tab '{INPUT_TAB_NAME}' not found.")

    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)

    # Get all rows. Row 1 is header. Row 2 is our first target.
    rows = ws_input.get_all_values()
    if len(rows) <= 1:
        return print("📭 Input Queue is empty.")

    # We process one by one and delete/update to keep the queue clean
    # We loop through a copy of the list so we don't mess up the indexing
    queries_to_process = rows[1:]

    all_menu_rows = []
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    print(f"🚀 Processing {len(queries_to_process)} entries...")

    for row_data in queries_to_process:
        query = row_data[0] # The Name + Address
        if not query:
            ws_input.delete_rows(2) # Remove empty rows
            continue

        print(f"🔎 Current Task: {query}")

        try:
            # 1. Search Google Maps
            headers = {
                "Content-Type": "application/json",
                "X-Goog-Api-Key": MAPS_API_KEY,
                "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
            }
            resp = requests.post("https://places.googleapis.com/v1/places:searchText",
                                 headers=headers, json={"textQuery": query}).json()

            places = resp.get('places', [])
            if not places:
                print(f"⚠️ Not found. Skipping.")
                ws_input.update_cell(2, 2, "FAILED: Not Found")
                # Move failed items to the bottom or keep them? Let's move to a "Failed" sheet or just skip.
                # For now, we'll just leave it and you can manually check.
                continue

            p = places[0]
            pid, name, loc = p.get('id'), p['displayName']['text'], p['formattedAddress']

            # 2. AI Enrichment
            ai = enrich_market_intelligence(name, loc)
            if not ai:
                ws_input.update_cell(2, 2, "FAILED: AI Error")
                continue

            d, menu_items = ai.get('details', {}), ai.get('menu', [])
            city, state, country = parse_address_components(p.get('addressComponents', []))

            # 3. Save to Main Sheets
            ws_rest.append_row([
                curr_time, pid, name, p.get('internationalPhoneNumber', 'N/A'),
                loc, city, state, country, d.get('cuisine'), d.get('website'),
                d.get('usp'), d.get('market_gap'), ", ".join(d.get('social_links', []))
            ])

            for m in menu_items:
                all_menu_rows.append([
                    str(uuid.uuid4()), pid, name, m.get('item_name'),
                    m.get('price'), m.get('category'), m.get('is_unique_locally'),
                    m.get('competitive_reasoning')
                ])

            # 4. Sync to Algolia
            algolia_client.save_object(index_name=ALG_INDEX_NAME, body={
                "objectID": pid,
                "name": name,
                "phone": p.get('internationalPhoneNumber', 'N/A'),
                "location": loc,
                "city": city,
                "country": country,
                "cuisine": d.get('cuisine'),
                "usp": d.get('usp'),
                "market_gap": d.get('market_gap'),
                "menu": menu_items
            })

            # 5. Cleanup: Delete the row from Input_Queue as it's finished
            ws_input.delete_rows(2)
            print(f"✅ Success! Row removed from Queue.")

            # Batch save menus every 5 restaurants to prevent data loss
            if len(all_menu_rows) > 10:
                ws_menu.append_rows(all_menu_rows)
                all_menu_rows = []

        except Exception as e:
            print(f"❌ Error processing {query}: {e}")
            ws_input.update_cell(2, 2, f"ERROR: {str(e)[:50]}")
            break # Stop if there's a major connection error

    # Final Menu Save
    if all_menu_rows:
        ws_menu.append_rows(all_menu_rows)

    print("\n✨ Batch Audit Session Finished.")

run_secure_batch_processor()

In [ ]:
# --- MULTI-LOCATION BATCH SCOUT ---
#@title 📡 Multi-Location Scout (Input from Sheet)
WORKBOOK_NAME = "V1 Advanced Market Intelligence 2025" #@param {type:"string"}
INPUT_TAB_NAME = "Loc_queue" #@param {type:"string"}
RADIUS_METERS = 7000 #@param {type:"slider", min:500, max:50000, step:500}

def run_multi_location_scout():
    print(f"📂 Accessing {WORKBOOK_NAME}...")
    sh = gc.open(WORKBOOK_NAME)

    try:
        ws_input = sh.worksheet(INPUT_TAB_NAME)
    except gspread.WorksheetNotFound:
        return print(f"❌ Tab '{INPUT_TAB_NAME}' not found.")

    ws_rest, ws_menu = get_sheets(WORKBOOK_NAME)

    # Get the list of locations to scout
    rows = ws_input.get_all_values()
    if len(rows) <= 1:
        return print("📭 No locations found in the queue.")

    locations_to_scout = rows[1:] # Skip header
    existing_ids = {row[1] for row in ws_rest.get_all_values()[1:]}
    curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

    for loc_row in locations_to_scout:
        location_name = loc_row[0]
        if not location_name:
            ws_input.delete_rows(2)
            continue

        print(f"\n🌍 SCOUTING AREA: {location_name}...")

        # 1. Geocoding the area
        geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                           params={"address": location_name, "key": MAPS_API_KEY}).json()

        if geo['status'] != 'OK':
            print(f"❌ Could not find location: {location_name}")
            ws_input.update_cell(2, 2, "FAILED: Invalid Location")
            continue

        lat, lng = geo['results'][0]['geometry']['location'].values()

        # 2. Search Nearby Restaurants
        headers = {
            "Content-Type": "application/json",
            "X-Goog-Api-Key": MAPS_API_KEY,
            "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.internationalPhoneNumber,places.addressComponents"
        }
        search_data = {
            "includedTypes": ["restaurant"],
            "maxResultCount": 15,
            "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": RADIUS_METERS}}
        }
        places_resp = requests.post("https://places.googleapis.com/v1/places:searchNearby",
                                    headers=headers, json=search_data).json()

        places = places_resp.get('places', [])
        print(f"📍 Found {len(places)} restaurants in {location_name}.")

        all_menu_rows = []

        # 3. Process each restaurant found in this area
        for p in places:
            pid, name, addr = p.get('id'), p['displayName']['text'], p['formattedAddress']

            if pid in existing_ids:
                continue

            print(f"🔎 Analyzing: {name}...")
            ai = enrich_market_intelligence(name, addr)
            if not ai: continue

            d, menu_items = ai.get('details', {}), ai.get('menu', [])
            city, state, country = parse_address_components(p.get('addressComponents', []))

            # Save Restaurant
            ws_rest.append_row([
                curr_time, pid, name, p.get('internationalPhoneNumber', 'N/A'),
                addr, city, state, country, d.get('cuisine'), d.get('website'),
                d.get('usp'), d.get('market_gap'), ", ".join(d.get('social_links', []))
            ])

            # Prepare Menus
            for m in menu_items:
                all_menu_rows.append([
                    str(uuid.uuid4()), pid, name, m.get('item_name'),
                    m.get('price'), m.get('category'), m.get('is_unique_locally'),
                    m.get('competitive_reasoning')
                ])

            # Algolia Sync
            try:
                algolia_client.save_object(index_name=ALG_INDEX_NAME, body={
                "objectID": pid,
                "name": name,
                "phone": p.get('internationalPhoneNumber', 'N/A'),
                "location": addr,
                "city": city,
                "country": country,
                "cuisine": d.get('cuisine'),
                "usp": d.get('usp'),
                "market_gap": d.get('market_gap'),
                "menu": menu_items
                })
            except: pass

            existing_ids.add(pid) # Track locally to prevent duplicates in same run
            time.sleep(0.5)

        # Batch save menus for this specific location
        if all_menu_rows:
            ws_menu.append_rows(all_menu_rows)

        # 4. Cleanup: Remove this location from the queue
        ws_input.delete_rows(2)
        print(f"✅ Finished {location_name}. Row removed.")

    print("\n✨ All locations scouted successfully!")

run_multi_location_scout()

In [ ]:
import requests
import json
import time
import uuid
import re
import gspread

# --- CONFIGURATION ---
#@title 📡 Food walk scout with cuisine type, diet pref
WALK_LOCATION = "Connaught Place, New Delhi" #@param {type:"string"}
WALK_NAME = "The Heart of Lutyens" #@param {type:"string"}
CUISINE_TYPE = "North Indian" #@param {type:"string"}
DIETARY_PREFERENCE = "Vegetarian" #@param ["Vegetarian", "Non-Vegetarian", "Any"]
ALG_WALK_INDEX = "prod_MENU_Walk"

def get_walking_duration(origin_id, dest_id):
    """Calculates walking time between two Google Place IDs."""
    url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        "origins": f"place_id:{origin_id}",
        "destinations": f"place_id:{dest_id}",
        "mode": "walking",
        "key": MAPS_API_KEY
    }
    try:
        data = requests.get(url, params=params).json()
        if data['rows'][0]['elements'][0]['status'] == 'OK':
            return data['rows'][0]['elements'][0]['duration']['text']
    except: return "5-10 mins walk"
    return "short walk"

def run_pro_food_walk():
    print(f"👟 Curating Walk: {WALK_NAME}...")
    sh = gc.open(WORKBOOK_NAME)

    # 1. Ensure Worksheet exists with all headers
    try:
        ws_walk = sh.worksheet("Food_Walks")
    except gspread.WorksheetNotFound:
        header = ['walk_id', 'walk_name', 'stop_no', 'rest_name', 'cuisine_type', 'is_veg_friendly',
                  'place_id', 'dish_to_try', 'price_est', 'google_price_level', 'walking_time_to_next',
                  'rest_address', 'locality', 'city', 'state', 'country',
                  'itinerary', 'is_open_now', 'map_link', 'timestamp']
        ws_walk = sh.add_worksheet("Food_Walks", 1000, len(header))
        ws_walk.append_row(header)

    # 2. Get Geolocation for Center Point
    geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                       params={"address": WALK_LOCATION, "key": MAPS_API_KEY}).json()
    lat, lng = geo['results'][0]['geometry']['location'].values()

    # 3. Get Data via SearchText (New API)
    text_query = f"{DIETARY_PREFERENCE} {CUISINE_TYPE} restaurants in {WALK_LOCATION}"

    headers = {
        "X-Goog-Api-Key": MAPS_API_KEY,
        "Content-Type": "application/json",
        # We must use 'places.id' and 'places.name' (which is the resource name)
        "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.addressComponents,places.priceLevel,places.types,places.servesVegetarianFood,places.rating"
    }

    search_data = {
        "textQuery": text_query,
        "locationBias": {
            "circle": {
                "center": {"latitude": lat, "longitude": lng},
                "radius": 2000
            }
        }
    }

    response = requests.post("https://places.googleapis.com/v1/places:searchText",
                             headers=headers, json=search_data)
    places = response.json().get('places', [])

    if not places:
        print("❌ No places found. Check your API Key or Query.")
        return

    # 4. AI Design Context
    restaurants_context = "\n".join([
        f"- {p['displayName']['text']} (ID: {p['id']}) [Rating: {p.get('rating', 'N/A')}]"
        for p in places
    ])

    prompt = (
        f"Design a 3-stop {DIETARY_PREFERENCE} {CUISINE_TYPE} food walk called '{WALK_NAME}' in {WALK_LOCATION}.\n"
        f"Available Spots:\n{restaurants_context}\n\n"
        "Return ONLY JSON: {'stops': [{'stop_no', 'place_id', 'rest_name', 'dish_to_try', 'price_est', 'itinerary'}]}"
    )

    try:
        ai_response = client.models.generate_content(model='gemini-2.0-flash', contents=prompt)
        walk_data = json.loads(re.search(r'\{.*\}', ai_response.text, re.DOTALL).group())
        stops = walk_data['stops']

        walk_id = str(uuid.uuid4())[:8]
        curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

        map_query = "/".join([s['rest_name'].replace(" ","+") for s in stops])
        full_map_link = f"https://www.google.com/maps/dir/{map_query}"

        final_rows = []
        for i, stop in enumerate(stops):
            # Walking duration
            walking_time = "Finish"
            if i < len(stops) - 1:
                walking_time = get_walking_duration(stop['place_id'], stops[i+1]['place_id'])

            # Match stop with Google data
            p_data = next((p for p in places if p['id'] == stop['place_id']), {})

            # --- EXTRACT DATA SAFELY ---
            # IMPORTANT: In New API, place_id is simply 'id'
            place_id_val = p_data.get('id', stop['place_id'])
            full_address = p_data.get('formattedAddress', 'N/A')
            components = p_data.get('addressComponents', [])

            def find_comp(type_list):
                for c in components:
                    if any(t in c.get('types', []) for t in type_list):
                        return c.get('longText', 'N/A')
                return "N/A"

            locality = find_comp(["locality", "sublocality_level_1"])
            city = find_comp(["administrative_area_level_2", "locality"])
            state = find_comp(["administrative_area_level_1"])
            country = find_comp(["country"])

            raw_types = p_data.get('types', [])
            cuisine = next((t.replace("_restaurant", "").replace("_", " ").title()
                           for t in raw_types if "restaurant" in t and t != "restaurant"), CUISINE_TYPE)
            is_veg = "Yes" if p_data.get('servesVegetarianFood') else "No"
            price_lvl = p_data.get('priceLevel', 'N/A').replace("PRICE_LEVEL_", "")

            # 5. Prepare Row for GSheets
            row = [
                walk_id, WALK_NAME, stop['stop_no'], stop['rest_name'], cuisine, is_veg,
                place_id_val, stop['dish_to_try'], stop['price_est'], price_lvl,
                walking_time, full_address, locality, city, state, country,
                stop['itinerary'], "Yes", full_map_link, curr_time
            ]
            final_rows.append(row)

            # 6. Sync to Algolia
            algolia_client.save_object(index_name=ALG_WALK_INDEX, body={
                "objectID": f"{walk_id}_{stop['stop_no']}",
                "walk_name": WALK_NAME,
                "walk_id": walk_id,
                "cuisine_type": cuisine,
                "is_veg_friendly": is_veg,
                "place_id": place_id_val,
                "rest_address": full_address,
                "locality": locality,
                "city": city,
                "state": state,
                "country": country,
                **stop
            })

        ws_walk.append_rows(final_rows)
        print(f"✅ Success! {WALK_NAME} curated with all location fields.")

    except Exception as e:
        print(f"❌ Error during processing: {e}")

run_pro_food_walk()

In [ ]:
import requests
import json
import time
import uuid
import re
import gspread

# --- CONFIGURATION ---
# --- MULTI- food walk scout ---
#@title 📡 Multi-Food walk batch Scout (Input from Sheet)
INPUT_TAB_NAME = "Walk_Queue"  #@param {type:"string"}
OUTPUT_TAB_NAME = "Food_Walks"  #@param {type:"string"}
ALG_WALK_INDEX = "prod_MENU_Walk"

def get_walking_duration(origin_id, dest_id):
    """Calculates walking time between two Google Place IDs."""
    url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        "origins": f"place_id:{origin_id}",
        "destinations": f"place_id:{dest_id}",
        "mode": "walking",
        "key": MAPS_API_KEY
    }
    try:
        data = requests.get(url, params=params).json()
        if data['rows'][0]['elements'][0]['status'] == 'OK':
            return data['rows'][0]['elements'][0]['duration']['text']
    except: return "5-10 mins walk"
    return "short walk"

def process_single_walk(sh, walk_input):
    """Core logic to curate one walk based on row input."""
    location = walk_input['WALK_LOCATION']
    name = walk_input['WALK_NAME']
    cuisine_req = walk_input['CUISINE_TYPE']
    dietary_req = walk_input['DIETARY_PREFERENCE']

    # 1. Get Geolocation
    geo = requests.get("https://maps.googleapis.com/maps/api/geocode/json",
                       params={"address": location, "key": MAPS_API_KEY}).json()
    if not geo.get('results'):
        print(f"❌ Could not find location: {location}")
        return False
    lat, lng = geo['results'][0]['geometry']['location'].values()

    # 2. Get Data via SearchText (New API)
    text_query = f"{dietary_req} {cuisine_req} restaurants in {location}"
    headers = {
        "X-Goog-Api-Key": MAPS_API_KEY,
        "Content-Type": "application/json",
        "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.addressComponents,places.priceLevel,places.types,places.servesVegetarianFood,places.rating"
    }
    search_data = {
        "textQuery": text_query,
        "locationBias": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": 2000}}
    }

    places_resp = requests.post("https://places.googleapis.com/v1/places:searchText",
                                headers=headers, json=search_data).json()
    places = places_resp.get('places', [])
    if not places:
        print(f"❌ No restaurants found for: {name}")
        return False

    # 3. AI Design
    restaurants_context = "\n".join([f"- {p['displayName']['text']} (ID: {p['id']})" for p in places[:10]])
    prompt = (
        f"Design a 3-stop {dietary_req} {cuisine_req} food walk called '{name}' in {location}.\n"
        f"Available Spots:\n{restaurants_context}\n\n"
        "Return ONLY JSON: {'stops': [{'stop_no', 'place_id', 'rest_name', 'dish_to_try', 'price_est', 'itinerary'}]}"
    )

    try:
        response = client.models.generate_content(model='gemini-2.0-flash', contents=prompt)
        walk_data = json.loads(re.search(r'\{.*\}', response.text, re.DOTALL).group())
        stops = walk_data['stops']

        walk_id = str(uuid.uuid4())[:8]
        curr_time = time.strftime("%Y-%m-%d %H:%M:%S")

        map_query = "/".join([s['rest_name'].replace(" ","+") for s in stops])
        full_map_link = f"https://www.google.com/maps/dir/{map_query}"

        final_rows = []
        for i, stop in enumerate(stops):
            walking_time = "Finish" if i == len(stops)-1 else get_walking_duration(stop['place_id'], stops[i+1]['place_id'])
            p_data = next((p for p in places if p['id'] == stop['place_id']), {})

            # Location Extraction
            comp = p_data.get('addressComponents', [])
            def find_c(ts): return next((c['longText'] for c in comp if any(t in c['types'] for t in ts)), "N/A")

            locality = find_c(["locality", "sublocality_level_1"])
            city = find_c(["administrative_area_level_2", "locality"])
            full_addr = p_data.get('formattedAddress', 'N/A')

            row = [
                walk_id, name, stop['stop_no'], stop['rest_name'], cuisine_req, dietary_req,
                p_data.get('id'), stop['dish_to_try'], stop['price_est'],
                p_data.get('priceLevel', 'N/A').replace("PRICE_LEVEL_", ""),
                walking_time, full_addr, locality, city,
                find_c(["administrative_area_level_1"]), find_c(["country"]),
                stop['itinerary'], "Yes", full_map_link, curr_time
            ]
            final_rows.append(row)

            # Sync to Algolia
            algolia_client.save_object(index_name=ALG_WALK_INDEX, body={
                "objectID": f"{walk_id}_{stop['stop_no']}",
                "walk_name": name, "walk_id": walk_id, "cuisine_type": cuisine_req,
                "is_veg_friendly": dietary_req, "rest_address": full_addr, "city": city, **stop
            })

        sh.worksheet(OUTPUT_TAB_NAME).append_rows(final_rows)
        return True

    except Exception as e:
        print(f"⚠️ Error processing {name}: {e}")
        return False

def run_batch_process():
    sh = gc.open(WORKBOOK_NAME)
    input_sheet = sh.worksheet(INPUT_TAB_NAME)

    # Check if Status column exists, if not, add it
    headers = input_sheet.row_values(1)
    if "Status" not in headers:
        input_sheet.update_cell(1, len(headers) + 1, "Status")
        input_sheet.update_cell(1, len(headers) + 2, "Processed_At")
        headers = input_sheet.row_values(1)

    status_col_idx = headers.index("Status") + 1
    time_col_idx = headers.index("Processed_At") + 1

    # Get data
    records = input_sheet.get_all_records()
    print(f"🚀 Found {len(records)} entries in queue.")

    for i, row in enumerate(records):
        # Skip if already completed
        if row.get("Status") == "Completed":
            continue

        if not row['WALK_NAME'] or not row['WALK_LOCATION']:
            continue

        success = process_single_walk(sh, row)

        if success:
            # Update the specific row in the input sheet (i+2 because GSheets is 1-indexed and has header)
            input_sheet.update_cell(i + 2, status_col_idx, "Completed")
            input_sheet.update_cell(i + 2, time_col_idx, time.strftime("%Y-%m-%d %H:%M:%S"))
            print(f"✅ Successfully curated: {row['WALK_NAME']}")
        else:
            input_sheet.update_cell(i + 2, status_col_idx, "Failed/No Data")

        time.sleep(2) # Prevent API rate limiting

run_batch_process()